# Sentiment Analysis Transformers

[Nicholas Renotte](https://www.youtube.com/watch?v=szczpgOEdXs)

## Install Dependencies

In [2]:
!pip3 install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.3 MB/s eta 0:12:21tcmalloc: large alloc 1147494400 bytes == 0x563ca703c000 @  0x7f7fc71cb615 0x563ca54244cc 0x563ca550447a 0x563ca54272ed 0x563ca5518e1d 0x563ca549ae99 0x563ca54959ee 0x563ca5428bda 0x563ca549ad00 0x563ca54959ee 0x563ca5428bda 0x563ca5497737 0x563ca5519c66 0x563ca5496daf 0x563ca5519c66 0x563ca5496daf 0x563ca5519c66 0x563ca5496daf 0x563ca5429039 0x563ca546c409 0x563ca5427c52 0x563ca549ac25 0x563ca54959ee 0x563ca5428bda 0x563ca5497737 0x563ca54959ee 0x563ca5428bda 0x563ca5496915 0x563ca5428afa 0x563ca5496c0d 0x563ca54959ee
     |██████████████████▌             | 1055.7 MB 1.3 MB/s eta 0:09:40tcmalloc: large alloc 1434370048 bytes == 0x563ceb692000 @  0x7f7fc71cb615 0x563ca54244cc 0x563ca550447a 0x563ca54272ed 0x563ca5518e1d 0x563ca549ae99 0x563ca54959ee 0x563ca5428bda 0x563ca549ad00 0x563ca54959ee 0x563ca5428bda 0x563ca5497737 0x563ca5519c66 0x563c

In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 2.9 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 36.5 MB/s 
     |████████████████████████████████| 3.3 MB 43.0 MB/s 
     |████████████████████████████████| 56 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 66.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Import Modules

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bs4 import BeautifulSoup
import requests
import torch
import re

## Initialize the Model

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

## Encode and Calculate Sentiment

In [14]:
tokens = tokenizer.encode("I love this food, this absolutely great tastes")

In [15]:
tokens

[101,
 151,
 11157,
 10372,
 15225,
 117,
 10372,
 35925,
 10563,
 11838,
 44960,
 10107,
 102]

In [18]:
tokens = tokenizer.encode("I love this food, this absolutely great tastes", return_tensors='pt')

In [19]:
tokens

tensor([[  101,   151, 11157, 10372, 15225,   117, 10372, 35925, 10563, 11838,
         44960, 10107,   102]])

In [20]:
tokens[0]

tensor([  101,   151, 11157, 10372, 15225,   117, 10372, 35925, 10563, 11838,
        44960, 10107,   102])

In [22]:
tokenizer.decode(tokens[0])

'[CLS] i love this food, this absolutely great tastes [SEP]'

In [23]:
result = model(tokens)

In [24]:
result

SequenceClassifierOutput([('logits',
                           tensor([[-2.2228, -2.6698, -1.3324,  1.1316,  4.2112]],
                                  grad_fn=<AddmmBackward0>))])

In [26]:
result.logits

tensor([[-2.2228, -2.6698, -1.3324,  1.1316,  4.2112]],
       grad_fn=<AddmmBackward0>)

In [28]:
int(torch.argmax(result.logits)) + 1

5

### Another example

In [29]:
tokens = tokenizer.encode("This is bad idea sir, the weather is to cold", return_tensors='pt')

In [30]:
result = model(tokens)

In [31]:
result.logits

tensor([[ 1.8752,  1.3616,  0.6572, -1.2122, -2.1315]],
       grad_fn=<AddmmBackward0>)

In [32]:
int(torch.argmax(result.logits)) + 1

1

## Collecting Review

In [45]:
r = requests.get('https://www.yelp.com/biz/akshar-home-made-food-colma')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [46]:
reviews

["It's always great to find home made vegetarian Indian food, and there should be really be more offering it. Akshar's cuisine is from Gujarat (nw India) and has a different flavor than I've had here and that's what makes Indian food a foodie experience.Akshar delivers tiffin (meals), about 7 dishes that will feed a family of 4 with plenty of leftovers. He says he'll even deliver to SF. I recommend!",
 'We had our first delivery today! \xa0My son, who is vegetarian, states this is his "new favorite Indian restaurant"! \xa0Really enjoyed the flavors, some of the dishes were familiar and others had different flavors from the standard fare. \xa0Price is reasonable - especially as 3 of us ate 2 meals today off of it and we still have a bit left over! \xa0Customer service is also exceptional - very nice and easy to deal with (we texted and I used Venmo to pay for the food). \xa0We like to cook ourselves, so won\'t be getting this weekly, but we will likely be back to order again frequently!

In [41]:
# r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
# soup = BeautifulSoup(r.text, 'html.parser')
# regex = re.compile('.*comment.*')
# results = soup.find_all('p', {'class':regex})
# reviews = [result.text for result in results]

In [48]:
reviews[0]

"It's always great to find home made vegetarian Indian food, and there should be really be more offering it. Akshar's cuisine is from Gujarat (nw India) and has a different flavor than I've had here and that's what makes Indian food a foodie experience.Akshar delivers tiffin (meals), about 7 dishes that will feed a family of 4 with plenty of leftovers. He says he'll even deliver to SF. I recommend!"

## Load Revies into DataFrame and Score

In [49]:
import pandas as pd
import numpy as np

In [51]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [52]:
df.head()

,review
0,It's always great to find home made vegetarian...
1,"We had our first delivery today! My son, who ..."


In [53]:
df['review'].iloc[0]

"It's always great to find home made vegetarian Indian food, and there should be really be more offering it. Akshar's cuisine is from Gujarat (nw India) and has a different flavor than I've had here and that's what makes Indian food a foodie experience.Akshar delivers tiffin (meals), about 7 dishes that will feed a family of 4 with plenty of leftovers. He says he'll even deliver to SF. I recommend!"

In [55]:
def sentiment_analysis(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1

In [56]:
sentiment_analysis(df['review'].iloc[0])

5

In [57]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_analysis(x[:512])) # How many tokens being converted

In [58]:
df

,review,sentiment
0,It's always great to find home made vegetarian...,5
1,"We had our first delivery today! My son, who ...",5


## Another

In [59]:
r = requests.get('https://www.yelp.com/biz/grand-prix-car-wash-san-pablo-san-pablo-3')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [60]:
reviews

["For the price, I thought they were terrific. \xa0 We got the basic Platinum for our cars for $45 each, which was then discounted. \xa0They have a military discount (20%), Senior Discount (15%) and an Early Bird ($5 off if you come in between 8 and 10 AM on a weekday).They missed a few spots (the trunk and part of the back window), but when I pointed that out, they cheerfully made it clean.I was happy AND it made it easy to sell the car. \xa0The new owner loves it. \xa0I'll be back.",
 "This is the worse car wash I've ever been to, there is no organization to this place. The only reason why your car looks washed is because it's been sitting out in the sun because their is no sense of urgency. I've seen cars come and get wash and leave while other cars hasn't been touched for over an hour. I understand that everyone is short staffed but you have to have a sense of someone knowing what they're doing. Everyone is trying to work but they just don't know what they're doing. Ridiculous, one

In [61]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [62]:
df.head()

,review
0,"For the price, I thought they were terrific. ..."
1,"This is the worse car wash I've ever been to, ..."
2,I wouldn't go back again also recommend you no...
3,Usually I come here and they have no problems ...
4,"Very dead car wash, 15 cars and still took ove..."


In [63]:
df.tail()

,review
5,"After a year, I treated my minivan to some det..."
6,"Very bad Severice, wait for more than an hours..."
7,I used to bring my car here a lot but I wouldn...
8,"Wow just dropped $120 for an ""interior vinyl c..."
9,Should merit a 5 star from the actual service ...


In [64]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_analysis(x[:512])) # How many tokens being converted

In [65]:
df

,review,sentiment
0,"For the price, I thought they were terrific. ...",4
1,"This is the worse car wash I've ever been to, ...",1
2,I wouldn't go back again also recommend you no...,1
3,Usually I come here and they have no problems ...,2
4,"Very dead car wash, 15 cars and still took ove...",2
5,"After a year, I treated my minivan to some det...",4
6,"Very bad Severice, wait for more than an hours...",1
7,I used to bring my car here a lot but I wouldn...,2
8,"Wow just dropped $120 for an ""interior vinyl c...",1
9,Should merit a 5 star from the actual service ...,5
